In [104]:
# Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

import asyncio

## 1. 이벤트 루프 (Event Loop): 오케스트라의 지휘자

In [151]:
async def helloWorld():
    await asyncio.sleep(0.25)
    print("[0.25s] Hello World")

async def loopStopAfter1Sec():
    await asyncio.sleep(0.5)
    print("[0.50s] Kill")
    loop.stop() 

loop = asyncio.new_event_loop()

# reservate tasks to be done
loop.create_task(helloWorld())
loop.create_task(loopStopAfter1Sec())

# run the loop
loop.run_forever()

[0.25s] Hello World
[0.50s] Kill


## 2. 비동기 함수와 코루틴 (Coroutines)

In [ ]:
# 일반적인 함수
def hello_printer():
    print("안녕, 나는 단순한 프린터야.")

hello_printer() # 이렇게 부르면 바로 실행됩니다.

안녕, 나는 단순한 프린터야.


In [ ]:
async def loudmouth_penguin(magic_number: int):
    print(f"나는 펭귄이야. 내 행운의 숫자는 {magic_number}지.")

coroutine = loudmouth_penguin(magic_number=3)
print(coroutine)

<coroutine object loudmouth_penguin at 0x7fda50443440>


/tmp/ipykernel_262170/2287150880.py:7: RuntimeWarning: coroutine 'loudmouth_penguin' was never awaited
  print(loudmouth_penguin(magic_number=3))


In [157]:
async def loudmouth_penguin(magic_number: int):
    print(f"나는 펭귄이야. 내 행운의 숫자는 {magic_number}지.")

coroutine = loudmouth_penguin(magic_number=3)

loop = asyncio.new_event_loop()
loop.run_until_complete(coroutine)
loop.close()

나는 펭귄이야. 내 행운의 숫자는 3지.


In [158]:
coroutine = loudmouth_penguin(magic_number=3)
# 1. 새로운 루프 생성
# 2. run_until_complete() 실행
# 3. 마무리 작업 및 자원 해제 (loop.close 포함)
asyncio.run(coroutine)

나는 펭귄이야. 내 행운의 숫자는 3지.


### **배경 지식: 제너레이터(Generator)**

In [126]:
def get_random_number():
    print("Hi")
    yield 1
    print("Hello")
    yield 7

# 실행하면 제너레이터 객체만 나옵니다.
generator = get_random_number()
print(next(generator)) # Hi 출력 후 1을 내놓고 멈춤
print(next(generator)) # Hello 출력 후 7을 내놓고 멈춤

Hi
1
Hello
7


## 3. 태스크 (Task): 이벤트 루프의 할 일 목록

In [161]:
loop = asyncio.new_event_loop()

coroutine = asyncio.sleep(1)
task = loop.create_task(coroutine)

try:
    loop.run_until_complete(task)
finally:
    loop.close()

In [ ]:
import asyncio

async def make_pasta():
    print("   [주방] 🍝 파스타 요리 시작...")
    await asyncio.sleep(1) 
    print("   [주방] 🍝 파스타 완성!")

async def test():
    
    # <class 'coroutine'>
    coroutine = make_pasta()
    
    # <class 'asyncio.tasks.Task'>
    task = asyncio.create_task(coroutine)
    
    await asyncio.sleep(2)
    print("[홀] 작업 완료!")

asyncio.run(test())

   [주방] 🍝 파스타 요리 시작...
   [주방] 🍝 파스타 완성!
[홀] 작업 완료!


### ⚠️ 주의: 가비지 컬렉션 (쓰레기 수거)의 함정

## 4. `await`: 기다림의 미학

In [225]:
async def make_pasta():
    print("   [주방] 🍝 파스타 시작 (3초)")
    await asyncio.sleep(3)
    print("   [주방] 🍝 파스타 끝")

async def restaurant():
    task = asyncio.create_task(make_pasta())
    await task
    
asyncio.run(restaurant())

   [주방] 🍝 파스타 시작 (3초)
   [주방] 🍝 파스타 끝


1. **Callback 등록**: `await task`가 실행되면, `task` 객체 내부에 "이 작업이 끝나면 `restaurant`를 다시 깨워줘(resume)"라는 **Callback**이 등록됩니다.

2. **제어권 양보 (Yielding Control):** `restaurant`는 멈추고, 제어권은 **이벤트 루프**로 넘어갑니다.

3. **다른 작업 수행**: 이벤트 루프는 `task`를 포함하여 대기 중인 다른 작업들을 수행할 수 있는 자유를 얻습니다.

4. **복귀**: `task`가 완료되면, 아까 등록된 콜백이 실행되어 `restaurant`가 다시 이벤트 루프의 대기열에 올라가고, 순서가 되면 실행을 재개합니다.

In [224]:
import asyncio

async def make_pasta():
    print("   [주방] 🍝 파스타 시작 (3초)")
    await asyncio.sleep(3)
    print("   [주방] 🍝 파스타 끝")

async def make_steak():
    print("   [주방] 🥩 스테이크 시작 (5초)")
    await asyncio.sleep(5)
    print("   [주방] 🥩 스테이크 끝")

async def restaurant():

    # 1. 두 개의 주문을 연달아 넣습니다. (아직 안 기다림!)
    task1 = asyncio.create_task(make_pasta())
    task2 = asyncio.create_task(make_steak())
    print("[홀] 두 가지 요리를 주문 넣었습니다. 기다립니다...")

    # 2. 이제 두 요리가 다 될 때까지 기다립니다.
    # 파스타가 끓는 동안 -> 스테이크도 굽고 있습니다. (동시 진행)
    await task1
    await task2
    print("[홀] 모든 요리가 나왔습니다!")

asyncio.run(restaurant())

[홀] 두 가지 요리를 주문 넣었습니다. 기다립니다...
   [주방] 🍝 파스타 시작 (3초)
   [주방] 🥩 스테이크 시작 (5초)
   [주방] 🍝 파스타 끝
   [주방] 🥩 스테이크 끝
[홀] 모든 요리가 나왔습니다!


많은 개발자가 `await coroutine()`을 하면 자동으로 다른 작업들과 병렬로 실행될 것이라고 착각하지만, 실제로는 그렇지 않습니다. Task로 감싸지 않은 순수 코루틴(Raw Coroutine)을 `await`하면, 제어권은 이벤트 루프로 돌아가지 않고 **다이렉트로 해당 코루틴으로 점프합니다.**

In [ ]:
import time

async def coro_a():
   time.sleep(0.5)
   print("나는 A야. 안녕!")

async def coro_b():
   print("나는 B야. 아무도 길 막지 말았으면...")

In [223]:
async def main():
   
   # task_b는 이벤트 루프에 등록되지만, 아직 실행 기회를 못 잡음
   task_b = asyncio.create_task(coro_b()) 

   # 여기서 제어권이 이벤트 루프로 안 넘어감!
   # coro_a()가 즉시 실행됨
   for _ in range(3):
      await coro_a()

   await task_b

asyncio.run(main())

나는 A야. 안녕!
나는 A야. 안녕!
나는 A야. 안녕!
나는 B야. 아무도 길 막지 말았으면...


In [222]:
async def main():
   task_b = asyncio.create_task(coro_b()) 

   # 태스크로 만들어서 await 함 -> 제어권이 이벤트 루프로 넘어감
   for _ in range(3):
      await asyncio.create_task(coro_a())

   await task_b

asyncio.run(main())

나는 B야. 아무도 길 막지 말았으면...
나는 A야. 안녕!
나는 A야. 안녕!
나는 A야. 안녕!
